# Formula 1 data analysis application season 2025
This is the 2025 version

This time, I use fastf1 for race data

<h2 style="color: red;">DON'T FORGET TO UPDATE RACE INDEX</h2>

In [ ]:
from racenameindex import RaceNameIndex

race_idx = RaceNameIndex.Australia
race_year = 2024

In [ ]:
# BackEnd
import fastf1 as ff1
import fastf1.plotting as ff1plt
import numpy as np
import pandas as pd
import plotly.express as px
from fastf1.core import Laps
from globaldata import update_data

# init
ff1plt.setup_mpl(mpl_timedelta_support=True, misc_mpl_mods=False, color_scheme='fastf1')

# last race data
race = ff1.get_session(race_year, race_idx, 'R')
race.load()
driver_laps = race.laps.pick_drivers(race.drivers)
driver_laps = driver_laps.reset_index()[driver_laps["IsAccurate"] == True] 
driver_laps["LapTime(s)"] = driver_laps["LapTime"].dt.total_seconds()
driver_laps["Time(s)"] = driver_laps["Time"].dt.total_seconds()
average_lap_time = driver_laps.groupby("Driver")["LapTime(s)"].mean().reset_index()
fastest = average_lap_time["LapTime(s)"].min()
average_lap_time['LapTimeDelta'] = average_lap_time['LapTime(s)'] - fastest
finishing_order = [race.get_driver(i)["Abbreviation"] for i in race.drivers]
average_lap_time["Position"] = average_lap_time["Driver"].apply(lambda x: finishing_order.index(x) + 1)
average_lap_time = average_lap_time.sort_values(by="Position")
average_lap_time["Driver"] = average_lap_time["Driver"].str.lower()

# qualifying data
quali_data = ff1.get_session(race_year, race_idx, 'Q')
quali_data.load()
quali_data = quali_data.laps.pick_drivers(quali_data.drivers)
quali_data = quali_data.reset_index()[quali_data["IsAccurate"] == True]
list_fastest_laps = list()
for drv in quali_data["Driver"].unique():
    drvs_fastest_lap = quali_data.pick_driver(drv).pick_fastest()
    list_fastest_laps.append(drvs_fastest_lap)
fastest_laps = Laps(list_fastest_laps) \
    .sort_values(by='LapTime') \
    .reset_index(drop=True)
pole_lap = fastest_laps.pick_fastest()
fastest_laps['LapTimeDelta'] = fastest_laps['LapTime'] - pole_lap['LapTime']
fastest_laps["LapTime"] = fastest_laps["LapTime"].dt.total_seconds()
fastest_laps["LapTimeDelta"] = fastest_laps["LapTimeDelta"].dt.total_seconds()
fastest_laps["Driver"] = fastest_laps["Driver"].str.lower()

# gap to winner
winner_lap = driver_laps[driver_laps["Position"] == 1].iloc[-1]
winner_driver = winner_lap["Driver"]
winner_times = driver_laps[driver_laps["Driver"] == winner_driver][["LapNumber", "Time(s)"]].set_index("LapNumber")
driver_laps = driver_laps.merge(winner_times, on="LapNumber", suffixes=("", "_Winner"))
driver_laps['GapToWinner'] = driver_laps['Time(s)'] - driver_laps['Time(s)_Winner']

# stints
stints = driver_laps[["Driver", "Stint", "Compound", "LapNumber"]]
stints = stints.groupby(["Driver", "Stint", "Compound"])
stints = stints.count().reset_index()
stints = stints.rename(columns={"LapNumber": "StintLength"})
stint_idxs = {drv: [1] for drv in stints["Driver"].unique()}
stints["Driver"] = stints["Driver"].str.lower()

# colors
colors = ff1plt.get_driver_color_mapping(session=race)
compound_colors = {}
for compound in stints["Compound"].unique():
    compound_colors[compound] = ff1plt.get_compound_color(compound, session=race)

# global year
global_df = update_data(race_year, race_idx)

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.4.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No ca

In [34]:
# get the average lap time per driver
px.scatter(average_lap_time, x="Driver", y="LapTime(s)", title="Average lap time per driver", template="plotly_dark")

In [6]:
# line plot of the lap times per driver per lap with color per driver
px.line(driver_laps, x="LapNumber", y="LapTime(s)", color="Driver", title="Lap times per driver per lap", template="plotly_dark", color_discrete_map=colors)

In [7]:
# position evolution per driver per lap (reverse y-axis)
fig = px.line(driver_laps, x="LapNumber", y="Position", color="Driver", title="Position evolution per driver", template="plotly_dark", color_discrete_map=colors)
fig.update_yaxes(autorange="reversed")
fig

In [ ]:
# type of tyre per driver per lap (horizontal bar plot)
fig = px.bar(stints, x="StintLength", y="Driver", color="Compound", title="Stint length per driver", template="plotly_dark", color_discrete_map=compound_colors, orientation="h")
fig.update_yaxes(tickmode="linear")
fig

In [17]:
# gap to winner
fig = px.line(driver_laps, x="LapNumber", y="GapToWinner", color="Driver",
              title="Écart temporel au vainqueur par pilote",
              template="plotly_dark", color_discrete_map=colors)
fig.update_yaxes(autorange="reversed")
fig

In [ ]:
# qualifying time
fig = px.scatter(fastest_laps, x="Driver", y="LapTime", title="Qualifying lap time delta to pole", template="plotly_dark", color="Driver", color_discrete_map=colors)

# lines constants
hundredsevenrule = pole_lap["LapTime"] * 1.07
hundredsevenrule = hundredsevenrule.total_seconds()
q1toq2 = 15
q2toq3 = 10

# add lines
fig.add_hline(y=hundredsevenrule, line_dash="dot", line_color="red", annotation_text="107% rule", annotation_position="bottom right") # 107% rule
fig.add_vline(x=q1toq2, line_dash="dot", line_color="red", annotation_text="eliminated in Q1", annotation_position="bottom right") # Q1 to Q2
fig.add_vline(x=q2toq3, line_dash="dot", line_color="red", annotation_text="eliminated in Q2", annotation_position="bottom right") # Q2 to Q3